# KEN3140 (Semantic Web) Lab 9
### Constructing ontologies with the OWL API

**Date:** 1 October 2020

**Author:** Kody Moodley

**License:** [GNU Lesser General Public License v3 (LGPL-3.0)](https://tldrlegal.com/license/gnu-lesser-general-public-license-v3-(lgpl-3))

**Notebook description:**

This is a Jupyter notebook to which allows one to programmatically construct ontologies using the [Java-based OWL API](https://github.com/owlcs/owlapi/wiki/Documentation) without requiring knowledge of the full API. We have created some wrapper classes and functions which allow you to make use of the most important features of the OWL API relevant for this course. Naturally, if you are an experienced Software Engineer or Java developer, you will be comfortable with making use of the full API. However, this lab will make use of the wrapper API we have setup which should be more accessible for those who are not as experienced with reading Java API documentation. You should be able to construct your ontologies quicker with more concise code using the wrappers defined in this notebook. **Note:** this notebook is designed for eductational purposes and is not intended to create production-scale ontologies. For that purpose, please use the full OWL API. 

**Instructions:**

Please read this section carefully before beginning the lab. Sections 1 - 3 of this notebook are purely for setting up the wrapper functions to allow you to build ontologies more concisely and quickly. Section 4 provides some example commands to show you how to construct classes, roles, individuals and axioms using most of the OWL 2 DL features. This section also shows how to load, save to file, and visualise your ontologies. Section 5 demonstrates how to execute reasoning tasks on your ontology. You can reuse code from Sections 4 and 5 if you wish in order to complete the lab task. From Section 6 onwards is where you will actually construct your **own** ontology using commands such as those in Sections 4 and 5.

**Requirements:**

* Jupyter notebooks: **https://jupyter.org/**
* IJava: **https://github.com/SpencerPark/IJava** (Java kernel for Jupyter notebooks)

**Documentation:**

For more details about how to use the OWL API, consult the documentation on Github: **https://github.com/owlcs/owlapi/wiki/Documentation**
The API (Java) docs for OWL API version 4 are located here: **http://owlcs.github.io/owlapi/apidocs_4/** and those for OWL API version 5 are located here: **http://owlcs.github.io/owlapi/apidocs_5/**
This notebook makes use of OWL API version 5 (specifically 5.1.2)

### 1. Import the OWL API
First we need to import the OWL API Java libraries and dependencies

In [1]:
%jars owlapi/lib/*.jar

In [2]:
%jars hermit/*.jar

In [3]:
import org.semanticweb.owlapi.apibinding.OWLManager;
import org.semanticweb.owlapi.model.*;
import uk.ac.manchester.cs.owl.owlapi.*;
import org.semanticweb.owlapi.expression.*;
import org.semanticweb.owlapi.util.*;
import org.semanticweb.owlapi.util.mansyntax.ManchesterOWLSyntaxParser;
import org.semanticweb.owlapi.manchestersyntax.parser.*;
import org.semanticweb.owlapi.manchestersyntax.renderer.*;
import org.semanticweb.owlapi.formats.*;
import java.io.FileOutputStream;
import java.util.stream.*;
import org.semanticweb.owlapi.model.parameters.*;
import org.semanticweb.HermiT.*;
import org.semanticweb.owlapi.reasoner.*;

### 2. Create a fresh ontology

In [4]:
// Ontology manager
OWLOntologyManager manager=OWLManager.createOWLOntologyManager();
// Ontology IRI
IRI ontologyIRI = IRI.create("http://maastrichtuniversity.nl/semweb/2020/lab9#");
// Create a fresh ontology
OWLOntology rootOntology=manager.createOntology(ontologyIRI);
// Create a data factory (ontology builder class)
OWLDataFactory df = new OWLDataFactoryImpl();
// entity renderer (prints ontology entities without IRI prefixes)
ManchesterOWLSyntaxOWLObjectRendererImpl m = new ManchesterOWLSyntaxOWLObjectRendererImpl();
// Initialise reasoner
Reasoner reasoner = new Reasoner(new Configuration(), rootOntology);

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


### 3. Wrapper functions & setup
Wrapper functions and classes to make it easier, more concise and quicker for students to use the OWL API to build ontologies

In [5]:
class Provider extends CachingBidirectionalShortFormProvider {

    private SimpleShortFormProvider provider = new SimpleShortFormProvider();

    @Override
    protected String generateShortForm(OWLEntity entity) {
        return provider.getShortForm(entity);
    }
}

OWLDataFactory df = OWLManager.getOWLDataFactory();
Provider shortFormProvider = new Provider();
OWLEntityChecker entityChecker = new ShortFormEntityChecker(shortFormProvider);
ManchesterOWLSyntaxParser parser = OWLManager.createManchesterParser();
parser.setOWLEntityChecker(entityChecker);

public void createClass(String classname) throws OWLOntologyCreationException, OWLOntologyStorageException {
    OWLClass c = df.getOWLClass(ontologyIRI.toString() + classname);
    shortFormProvider.add(c);
    OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(c, new HashSet<OWLAnnotation>());
    OWLAxiom class_declaration = a.getAxiomWithoutAnnotations();
    manager.addAxiom(rootOntology, class_declaration);
}
                          
public void createRole(String rolename, int trans, int ref, int sym) throws OWLOntologyCreationException, OWLOntologyStorageException {
    if (trans == 1){
        OWLObjectProperty r = df.getOWLObjectProperty(ontologyIRI.toString() + rolename);
        shortFormProvider.add(r);
        //OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(r, new HashSet<OWLAnnotation>());
        OWLTransitiveObjectPropertyAxiom t = df.getOWLTransitiveObjectPropertyAxiom(r, new HashSet<OWLAnnotation>());
        //OWLAxiom role_declaration = a.getAxiomWithoutAnnotations();
        manager.addAxiom(rootOntology, t);
    }
    if (ref == 1){
        OWLObjectProperty r = df.getOWLObjectProperty(ontologyIRI.toString() + rolename);
        shortFormProvider.add(r);
        //OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(r, new HashSet<OWLAnnotation>());
        OWLReflexiveObjectPropertyAxiom t = df.getOWLReflexiveObjectPropertyAxiom(r, new HashSet<OWLAnnotation>());
        //OWLAxiom role_declaration = a.getAxiomWithoutAnnotations();
        manager.addAxiom(rootOntology, t);
    }
    if (ref == 2){
        OWLObjectProperty r = df.getOWLObjectProperty(ontologyIRI.toString() + rolename);
        shortFormProvider.add(r);
        //OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(r, new HashSet<OWLAnnotation>());
        OWLIrreflexiveObjectPropertyAxiom t = df.getOWLIrreflexiveObjectPropertyAxiom(r, new HashSet<OWLAnnotation>());
        //OWLAxiom role_declaration = a.getAxiomWithoutAnnotations();
        manager.addAxiom(rootOntology, t);
    }
    if (sym == 1){
        OWLObjectProperty r = df.getOWLObjectProperty(ontologyIRI.toString() + rolename);
        shortFormProvider.add(r);
        //OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(r, new HashSet<OWLAnnotation>());
        OWLSymmetricObjectPropertyAxiom t = df.getOWLSymmetricObjectPropertyAxiom(r, new HashSet<OWLAnnotation>());
        //OWLAxiom role_declaration = a.getAxiomWithoutAnnotations();
        manager.addAxiom(rootOntology, t);
    }
    if (sym == 2){
        OWLObjectProperty r = df.getOWLObjectProperty(ontologyIRI.toString() + rolename);
        shortFormProvider.add(r);
        //OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(r, new HashSet<OWLAnnotation>());
        OWLAsymmetricObjectPropertyAxiom t = df.getOWLAsymmetricObjectPropertyAxiom(r, new HashSet<OWLAnnotation>());
        //OWLAxiom role_declaration = a.getAxiomWithoutAnnotations();
        manager.addAxiom(rootOntology, t);
    }
    if ((trans == 0) && (ref == 0) && (sym == 0)) {
        OWLObjectProperty r = df.getOWLObjectProperty(ontologyIRI.toString() + rolename);
        shortFormProvider.add(r);
        OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(r, new HashSet<OWLAnnotation>());
        OWLAxiom role_declaration = a.getAxiomWithoutAnnotations();
        manager.addAxiom(rootOntology, role_declaration);
    }
}
                          
public void createIndividual(String individualname) throws OWLOntologyCreationException, OWLOntologyStorageException {
    OWLNamedIndividual i = df.getOWLNamedIndividual(ontologyIRI.toString() + individualname);
    shortFormProvider.add(i);
    OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(i, new HashSet<OWLAnnotation>());
    OWLAxiom individual_declaration = a.getAxiomWithoutAnnotations();
    manager.addAxiom(rootOntology, individual_declaration);
}

public OWLAxiom createAxiom(String axiomStr) throws OWLOntologyCreationException, OWLOntologyStorageException {
    parser.setStringToParse(axiomStr);
    OWLAxiom axiom = parser.parseAxiom();
    manager.addAxiom(rootOntology, axiom);
    return axiom;
}

public OWLClassExpression createClassExpression(String classExpressionStr) throws OWLOntologyCreationException, OWLOntologyStorageException {
    parser.setStringToParse(classExpressionStr);
    OWLClassExpression clsEx = parser.parseClassExpression();
    return clsEx;
}

public OWLAxiom createRoleAssertion(String axiomStr) throws OWLOntologyCreationException, OWLOntologyStorageException {
    String [] parts = axiomStr.split(" ");
    if (parts.length < 3){
        System.out.println("Parser error: incorrect syntax for role assertion. requires exactly three tokens separated by single spaces");
        return null;
    }
    else{
        OWLObjectPropertyAssertionAxiom a = df.getOWLObjectPropertyAssertionAxiom(df.getOWLObjectProperty(ontologyIRI.toString() + parts[1]), df.getOWLNamedIndividual(ontologyIRI.toString() + parts[0]), df.getOWLNamedIndividual(ontologyIRI.toString() + parts[2]));
        manager.addAxiom(rootOntology, a);
        return a;
    }
}

public OWLAxiom allIndividualsDifferent(){
    Set<OWLIndividual> inds = new HashSet<OWLIndividual>();
    rootOntology.individualsInSignature(Imports.EXCLUDED).forEach(c->inds.add(c));
    
    if (inds.size() < 2){
        System.out.println("Error: requires more than 1 individual name in the ontology.");
        return null;
    }
    else{
        OWLAxiom a = df.getOWLDifferentIndividualsAxiom(inds, new HashSet<OWLAnnotation>()); 
        manager.addAxiom(rootOntology, a);
        return a;
    }
}

public OWLAxiom different(String differentIndividualsStr){
    String [] parts = differentIndividualsStr.split(" ");
    Set<OWLIndividual> inds = new HashSet<OWLIndividual>();
    for (String s: parts){
        inds.add(df.getOWLNamedIndividual(IRI.create(ontologyIRI.toString() + s)));
    }
    
    if (parts.length < 2){
        System.out.println("Parser error: incorrect syntax for different individuals axiom. requires more than 1 token (individual name) separated by single spaces");
        return null;
    }
    else{
        OWLAxiom a = df.getOWLDifferentIndividualsAxiom(inds, new HashSet<OWLAnnotation>()); 
        manager.addAxiom(rootOntology, a);
        return a;
    }
}

public void getSubClasses(String classEx) throws OWLOntologyCreationException, OWLOntologyStorageException {
    reasoner.flush();
    NodeSet<OWLClass> subclasses = reasoner.getSubClasses(createClassExpression(classEx));
    for (Node<OWLClass> nc: subclasses){
        for (OWLClass c: nc){
            if (!c.isOWLNothing() && !c.isOWLThing())
                System.out.println(m.render(c));
        }
    }
}

public void getSuperClasses(String classEx) throws OWLOntologyCreationException, OWLOntologyStorageException {
    reasoner.flush();
    NodeSet<OWLClass> supclasses = reasoner.getSuperClasses(createClassExpression(classEx));
    for (Node<OWLClass> nc: supclasses){
        for (OWLClass c: nc){
            if (!c.isOWLNothing() && !c.isOWLThing())
                System.out.println(m.render(c));
        }
    }
}

public void getUnsatisfiableClasses(){
    reasoner.flush();
    Node<OWLClass> classes = reasoner.getUnsatisfiableClasses();
    for (OWLClass c: classes){
        if (!c.isOWLNothing())
            System.out.println(m.render(c));
    }
}

public void isConsistent(){
    reasoner.flush();
    if (reasoner.isConsistent())
        System.out.println("Yes");
    else
        System.out.println("No");
}

public void print(OWLOntology ontology){
    Stream<OWLEntity> signature = ontology.signature(Imports.EXCLUDED);
    Stream<OWLAxiom> rbox = ontology.rboxAxioms(Imports.EXCLUDED);
    Stream<OWLAxiom> tbox = ontology.tboxAxioms(Imports.EXCLUDED);
    Stream<OWLAxiom> abox = ontology.aboxAxioms(Imports.EXCLUDED);
    
    Set<OWLEntity> clses = new HashSet<OWLEntity>();
    Set<OWLEntity> roles = new HashSet<OWLEntity>();
    Set<OWLEntity> inds = new HashSet<OWLEntity>();
    signature.forEach(e -> {
        if (e.isOWLClass()) clses.add(e);
        if (e.isOWLObjectProperty()) roles.add(e);
        if (e.isIndividual()) inds.add(e);
    });
    
    System.out.println();
    
    System.out.println("Classes:");
    System.out.println("--------");
    
    for (OWLEntity c: clses){
        System.out.println(m.render(c));
    }
    
    System.out.println();
    
    System.out.println("Roles:");
    System.out.println("------");

    for (OWLEntity r: roles){
        System.out.println(m.render(r));
    }
    
    System.out.println();
    
    System.out.println("Individuals:");
    System.out.println("------------");
    
    for (OWLEntity i: inds){
        System.out.println(m.render(i));
    }

    System.out.println();

    System.out.println("TBox:");
    System.out.println("-----");

    tbox.forEach(a -> System.out.println(m.render(a)));
    
    System.out.println();
    
    System.out.println("ABox:");
    System.out.println("-----");

    abox.forEach(a -> System.out.println(m.render(a)));    
    
    System.out.println();

    System.out.println("RBox:");
    System.out.println("-----");
    
    rbox.forEach(a -> System.out.println(m.render(a)));  
}

public void print(OWLAxiom axiom){
    System.out.println(m.render(axiom));
}

public void removeClass(String classname) throws OWLOntologyCreationException, OWLOntologyStorageException {
    OWLClass c = df.getOWLClass(ontologyIRI.toString() + classname);
    OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(c, new HashSet<OWLAnnotation>());
    manager.removeAxiom(rootOntology, a);
}

public void removeRole(String rolename) throws OWLOntologyCreationException, OWLOntologyStorageException {
    OWLObjectProperty r = df.getOWLObjectProperty(ontologyIRI.toString() + rolename);
    OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(r, new HashSet<OWLAnnotation>());
    manager.removeAxiom(rootOntology, a);
}

public void removeIndividual(String individualname) throws OWLOntologyCreationException, OWLOntologyStorageException {
    OWLNamedIndividual i = df.getOWLNamedIndividual(ontologyIRI.toString() + individualname);
    OWLDeclarationAxiomImpl a = new OWLDeclarationAxiomImpl(i, new HashSet<OWLAnnotation>());
    manager.removeAxiom(rootOntology, a);
}

public void removeAxiom(OWLAxiom a) throws OWLOntologyCreationException, OWLOntologyStorageException {
    manager.removeAxiom(rootOntology, a);
}

public void resetOntology(){
    rootOntology.removeAxioms(rootOntology.getAxioms());
}

public void saveOntology(String filepath) throws OWLOntologyStorageException, FileNotFoundException{
    FileOutputStream fout=new FileOutputStream(filepath);    
    manager.saveOntology(rootOntology, new ManchesterSyntaxDocumentFormat(), fout);
}

public void printStatistics(OWLOntology ontology){
    System.out.println();
    System.out.println("Ontology statistics:");
    System.out.println("--------------------");
    // Number of axioms and constructs in ontology
    System.out.println("Number of axioms: " + ontology.getAxiomCount());
    System.out.println("Number of classes: " + ontology.getClassesInSignature(true).size());
    System.out.println("Number of relations: " + ontology.getObjectPropertiesInSignature(true).size());
    System.out.println("Number of instances: " + ontology.getIndividualsInSignature(true).size());
    // Number of axioms of a specific type in ontology
    System.out.println("Number of SubClassOf axioms: " + ontology.getAxioms(AxiomType.SUBCLASS_OF).size());
    System.out.println("Number of EquivalentClasses axioms: " + ontology.getAxioms(AxiomType.EQUIVALENT_CLASSES).size());
    System.out.println("Number of DisjointClasses axioms: " + ontology.getAxioms(AxiomType.DISJOINT_CLASSES).size());
    System.out.println("Number of Class assertions: " + ontology.getAxioms(AxiomType.CLASS_ASSERTION).size());
    System.out.println("Number of ObjectProperty assertions: " + ontology.getAxioms(AxiomType.OBJECT_PROPERTY_ASSERTION).size());
}

### 4. Examples to get started
Have a look at these examples to show you how to build ontologies using this "wrapper" API I created for the OWL API

#### 4a. Creating (atomic) classes

In [6]:
// create class **names**
// You can only create **atomic** classes with the method below.
// For complex class expressions, you can create axioms (i.e. using the createAxiom method) 
// that defines or describes the relationship of this expression with others in the ontology.
// E.g. (Person and (hasChild some (hasChild some Person ))) defines "grandparents"
// But you would not just add this expression by itself to the ontology
// Generally, you want to express a logical statement using this expression:
// E.g. Parent equivalentTo (Person and (hasChild some (hasChild some Person )))
createClass("Woman");
createClass("Man");
createClass("Male");
createClass("Female");
createClass("Person");
createClass("Adult");
createClass("Car");
createClass("Tyre");
createClass("USCitizen");
createClass("Passport");
createClass("Leg");
createClass("Arthropod");
createClass("JohnsBookClubMembers");
print(rootOntology);


Classes:
--------
JohnsBookClubMembers
Man
Arthropod
Male
Car
Passport
Leg
USCitizen
Tyre
Person
Adult
Female
Woman

Roles:
------

Individuals:
------------

TBox:
-----

ABox:
-----

RBox:
-----


#### 4b. Creating (atomic) roles

In [7]:
// create roles
// parameter1: name of the role
// parameter2: transitive? 0 - no, 1 - yes
// parameter3: reflexive? 0 - no, 1 - yes, 2 - irreflexive
// parameter4: symmetric? 0 - no, 1 - yes, 2 - antisymmetric
createRole("hasGender", 0, 0, 0);
createRole("marriedTo", 0, 2, 1);
createRole("hasPart", 0, 0, 0);
createRole("teaches", 0, 0, 0);
createRole("studentOf", 0, 0, 0);
createRole("owns", 0, 0, 0);
createRole("hasSister", 0, 0, 0);
createRole("hasSibling", 0, 0, 0);
createRole("hasDocument", 0, 0, 0);
print(rootOntology);


Classes:
--------
JohnsBookClubMembers
Man
Arthropod
Male
Car
Passport
Leg
USCitizen
Tyre
Person
Adult
Female
Woman

Roles:
------
owns
hasDocument
hasGender
hasPart
teaches
hasSibling
studentOf
hasSister
marriedTo

Individuals:
------------

TBox:
-----

ABox:
-----

RBox:
-----
 Symmetric: marriedTo
 Irreflexive: marriedTo


#### 4c. Creating individuals

In [8]:
// create individuals
createIndividual("john");
createIndividual("sara");
createIndividual("ford_mondeo_16");
createIndividual("mary");
print(rootOntology);


Classes:
--------
JohnsBookClubMembers
Man
Arthropod
Male
Car
Passport
Leg
USCitizen
Tyre
Person
Adult
Female
Woman

Roles:
------
owns
hasDocument
hasGender
hasPart
teaches
hasSibling
studentOf
hasSister
marriedTo

Individuals:
------------
john
ford_mondeo_16
mary
sara

TBox:
-----

ABox:
-----

RBox:
-----
 Symmetric: marriedTo
 Irreflexive: marriedTo


#### 4d. Creating TBox axioms

In [9]:
// TBox axioms
// create subclass axioms
OWLAxiom subAx1 = createAxiom("Man subClassOf Person and (hasGender some Male)");
OWLAxiom subAx2 = createAxiom("Woman subClassOf Person and (hasGender some Female)");
// create equivalence axioms
OWLAxiom eqAx1 = createAxiom("Man equivalentTo Person and Adult and (hasGender only Male)");
OWLAxiom eqAx2 = createAxiom("Woman equivalentTo Person and Adult and (hasGender only Female)");
// create disjointness axioms
OWLAxiom disjAx1 = createAxiom("Man disjointWith Woman");
OWLAxiom disjAx2 = createAxiom("Female disjointWith Male");
// create axioms using cardinality restrictions
OWLAxiom cardAx1 = createAxiom("Car subClassOf hasPart exactly 4 Tyre");
OWLAxiom cardAx2 = createAxiom("USCitizen subClassOf hasDocument max 2 Passport");
OWLAxiom cardAx3 = createAxiom("Arthropod subClassOf hasPart min 6 Leg");
// create axioms using nominals
OWLAxiom nAx1 = createAxiom("JohnsBookClubMembers equivalentTo {john,mary,sara}");
//print(rootOntology);
// create domain & range axioms
OWLAxiom nAx1 = createAxiom("hasGender Domain: Person");
OWLAxiom nAx1 = createAxiom("hasGender Range: Male or Female");
print(rootOntology);

//System.out.println(reasoner.isConsistent());


Classes:
--------
JohnsBookClubMembers
Man
Arthropod
Male
Car
Passport
Leg
USCitizen
Tyre
Person
Adult
Female
Woman

Roles:
------
owns
hasDocument
hasGender
hasPart
teaches
hasSibling
studentOf
hasSister
marriedTo

Individuals:
------------
john
ford_mondeo_16
mary
sara

TBox:
-----
Man EquivalentTo Adult and Person and (hasGender only Male)
Woman EquivalentTo Adult and Person and (hasGender only Female)
JohnsBookClubMembers EquivalentTo {john , mary , sara}
Man SubClassOf Person and (hasGender some Male)
Woman SubClassOf Person and (hasGender some Female)
Car SubClassOf hasPart exactly 4 Tyre
USCitizen SubClassOf hasDocument max 2 Passport
Arthropod SubClassOf hasPart min 6 Leg
hasGender Range Female or Male
hasGender Domain Person
Man DisjointWith Woman
Female DisjointWith Male

ABox:
-----

RBox:
-----
 Symmetric: marriedTo
 Irreflexive: marriedTo


#### 4e. Creating ABox axioms

In [10]:
// ABox axioms
// create class assertion
OWLAxiom caAx1 = createAxiom("john Type: Man");
// **NB: create role assertion**
// With role assertions the method name is different
// **we cannot use createAxiom here**
// We use createRoleAssertion instead
OWLAxiom raAx1 = createRoleAssertion("john marriedTo mary");
// Create different individuals axiom (specify each individual name)
//OWLAxiom diAx1 = createAxiom("DifferentIndividuals: john mary sara");
OWLAxiom diAx1 = different("john mary sara");
// Make **all** individual names in the ontology refer to distinct objects from the domain
//OWLAxiom allDiff = allIndividualsDifferent();
print(rootOntology);


Classes:
--------
JohnsBookClubMembers
Man
Arthropod
Male
Car
Passport
Leg
USCitizen
Tyre
Person
Adult
Female
Woman

Roles:
------
owns
hasDocument
hasGender
hasPart
teaches
hasSibling
studentOf
hasSister
marriedTo

Individuals:
------------
john
ford_mondeo_16
mary
sara

TBox:
-----
Man EquivalentTo Adult and Person and (hasGender only Male)
Woman EquivalentTo Adult and Person and (hasGender only Female)
JohnsBookClubMembers EquivalentTo {john , mary , sara}
Man SubClassOf Person and (hasGender some Male)
Woman SubClassOf Person and (hasGender some Female)
Car SubClassOf hasPart exactly 4 Tyre
USCitizen SubClassOf hasDocument max 2 Passport
Arthropod SubClassOf hasPart min 6 Leg
hasGender Range Female or Male
hasGender Domain Person
Man DisjointWith Woman
Female DisjointWith Male

ABox:
-----
john Type Man
 DifferentIndividuals: john, mary, sara
john marriedTo mary

RBox:
-----
 Symmetric: marriedTo
 Irreflexive: marriedTo


#### 4f. Creating RBox axioms

In [11]:
// RBox axioms
// **note the ":" after SubPropertyOf and InverseOf**
// role hierarchies 
OWLAxiom roleHAx1 = createAxiom("hasSister SubPropertyOf: hasSibling");
// role compositions
OWLAxiom roleCAx1 = createAxiom("owns o hasPart SubPropertyOf: owns");
// inverse roles
OWLAxiom invAx1 = createAxiom("teaches InverseOf: studentOf");
print(rootOntology);
printStatistics(rootOntology);


Classes:
--------
JohnsBookClubMembers
Man
Arthropod
Male
Car
Passport
Leg
USCitizen
Tyre
Person
Adult
Female
Woman

Roles:
------
owns
hasDocument
hasGender
hasPart
teaches
hasSibling
studentOf
hasSister
marriedTo

Individuals:
------------
john
ford_mondeo_16
mary
sara

TBox:
-----
Man EquivalentTo Adult and Person and (hasGender only Male)
Woman EquivalentTo Adult and Person and (hasGender only Female)
JohnsBookClubMembers EquivalentTo {john , mary , sara}
Man SubClassOf Person and (hasGender some Male)
Woman SubClassOf Person and (hasGender some Female)
Car SubClassOf hasPart exactly 4 Tyre
USCitizen SubClassOf hasDocument max 2 Passport
Arthropod SubClassOf hasPart min 6 Leg
hasGender Range Female or Male
hasGender Domain Person
Man DisjointWith Woman
Female DisjointWith Male

ABox:
-----
john Type Man
 DifferentIndividuals: john, mary, sara
john marriedTo mary

RBox:
-----
hasSister SubPropertyOf: hasSibling
owns o hasPart SubPropertyOf: owns
 Symmetric: marriedTo
teaches Invers

#### 4g. Saving ontology to file

In [12]:
// // Save the ontology
// saveOntology("ontologyfilename.owl");
// // If you want to "reset" the ontology
// // I.e., remove all axioms from the ontology
// // Use the following command to start afresh
// resetOntology();
// // print the ontology
// print(rootOntology);

#### 4h. What if I want to load an ontology from a local file?

In [13]:
// // Obtain a reference to a local OWL ontology file
// File file = new File("ontologyfilename.owl");
// // **NB:** you can't store two ontologies with the same IRI in the same ontology manager instance
// // Create a new ontology manager
// OWLOntologyManager newManager = OWLManager.createOWLOntologyManager();
// // Load the ontology from file into the **new** ontology manager
// OWLOntology another_ontology = newManager.loadOntologyFromOntologyDocument(file);
// print(another_ontology);
// // some statistics about the ontology entities and axioms
// printStatistics(another_ontology);

#### 4i. ...or load one from a remote URI (hosted on the Web)

In [14]:
// // Specify the IRI where the ontology is located
// IRI remoteOntologyIRI = IRI.create("https://protege.stanford.edu/ontologies/pizza/pizza.owl");
// // Load the ontology from the URI into the ontology manager
// OWLOntology remote_ontology = manager.loadOntology(remoteOntologyIRI);
// print(remote_ontology);
// printStatistics(remote_ontology);

#### 4j. Visualise your ontology using [WebVowel](http://www.visualdataweb.de/webvowl/)
You can upload your OWL file (less than 5MB) to the WebVowel online ontology visualiser to explore your ontology graphically. Protege also has ontology visualisation plugins e.g. [OntoGraf](https://github.com/protegeproject/ontograf) (you can find this under Window>Tabs>OntoGraf. However, WebVowel is a more visually appealing interface.

![title](img/visualise_your_ont.png)

#### 5. Reasoning with your ontology using OWL 2 DL reasoners
Here we will use [HermiT](http://www.hermit-reasoner.com/) which is a robust OWL 2 DL reasoner. There is also [Pellet](https://github.com/stardog-union/pellet) and [Fact++](https://bitbucket.org/dtsarkov/factplusplus/downloads/uk.ac.manchester.cs.owl.factplusplus-P5.x-v1.6.5.jar) as alternatives amongst [others](https://www.w3.org/2001/sw/wiki/OWL/Implementations)

#### 5a. Classify the ontology 
Computes all subclass / superclass relationships between all atomic concepts in the ontology

In [15]:
reasoner.precomputeInferences(InferenceType.CLASS_HIERARCHY);
getSuperClasses("hasGender some Male");
System.out.println();
getSubClasses("hasGender some Male");

Person

Man


#### 5b. Is the ontology consistent?

In [16]:
createClass("Bird");
createClass("Penguin");
createClass("FlyingOrganism");
createAxiom("Bird subClassOf FlyingOrganism");
createAxiom("Penguin subClassOf Bird");
createAxiom("Penguin subClassOf not FlyingOrganism");
createIndividual("tweety");
OWLAxiom tweetyAxiom = createAxiom("tweety Type: Penguin");
isConsistent();

No


#### 5c. Compute all unsatisfiable classes

In [18]:
removeAxiom(tweetyAxiom);
getUnsatisfiableClasses();

Penguin


#### 5c. TBox subsumption

#### 5d. ABox realisation

#### 5e. Compute justifications